In [ ]:
# define imports
import numpy as np
import numpy.random
import pandas as pd
import random

import xml.etree.ElementTree as ET
#rd kit
from rdkit.Chem import Draw, AllChem, MACCSkeys
from rdkit import Chem, DataStructs
from rdkit.Chem.Fingerprints import FingerprintMols
#sklearn
from sklearn import datasets
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef
#importing the preprocessed embedds
embedds = dict( np.load('protein.npz', mmap_mode='r' ))

# Import DataSet from xml file

In [ ]:
# create DataFrame with predefined column names
DataSet = pd.DataFrame(columns=['name', 'smiles','molecule', 'fingerprint', 'target', 'SeqVec'])

In [ ]:
# parse XML
root = ET.parse('full_database.xml').getroot()
neg_targets = []

In [ ]:
# counter for stats
counter = 0

#for all drugs in databank
for i in range(len(root)):
    
    #if drug is a "small molecule"
    if (root[i].attrib.get('type') == "small molecule"):
        
        Datasetentry = []
        #tnumber counts the number of targets the drug has
        tarnumber = 0
        
        #getting the drugs name
        Datasetentry.append(root[i].find('{http://www.drugbank.ca}name').text)  
        
         
        #go thru properties searching for the smiles
        smiles = ''
        for property in root[i].find('{http://www.drugbank.ca}calculated-properties'):
            if(property[0].text == "SMILES"):
                smiles = property.find('{http://www.drugbank.ca}value').text
                Datasetentry.append(smiles)
        
        #add "X" if no smiles is found
        if len(Datasetentry) == 1:
            Datasetentry.append("X")
        
        
        #add empty cell for molecule object
        Datasetentry.append("0")
        
        #add empty cell for fingerprint object
        Datasetentry.append("0")
        
        
        
        #go thru targets to find their id
        try:
            #if the drug has a target its id is added to the dataframe
            if root[i].find('{http://www.drugbank.ca}targets').find('{http://www.drugbank.ca}target') != None:
                tars = root[i].find('{http://www.drugbank.ca}targets')
                for target in tars:
                    tar = target.find('{http://www.drugbank.ca}polypeptide')
                    x = tar.get('id')
                    Datasetentry.append(x)
                    Datasetentry.append("0")
                    DataSet.loc[len(DataSet)] = Datasetentry
                    Datasetentry = Datasetentry[:-2]
                    counter = counter + 1
                    tarnumber = tarnumber + 1
            #if it hasn't an empty cell is added instead
            else:
                Datasetentry.append("0")
                Datasetentry.append("0")
                DataSet.loc[len(DataSet)] = Datasetentry
        except:
            continue
    
        #adding the smiles of drugs that have between 3 and 10 targets to a list
        if tarnumber>2 and tarnumber<11:
            neg_targets.append(smiles)
        
        
print(len(DataSet))
print(counter)
print(len(neg_targets))

In [ ]:
#saving the dataset so we dont have to run the timeconsuming parts
pd.to_pickle(DataSet, "DataSet_SVM_woreduction", compression='infer', protocol=4)

In [ ]:
#loading the dataset from pickle
DataSet = pd.read_pickle("DataSet2_fpsize1k", compression='infer')

# Reduction of proteins

In [ ]:
from Bio import SeqIO
identifiers50 = []
with open('protein50.fasta') as fasta_file:  # Will close handle cleanly
    for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
        identifiers50.append(seq_record.id)

In [ ]:
protIds50 = []
for i in identifiers50:
    protIds50.append(i[i.find("|")+1:])

# Reduction of drugs

In [ ]:
def ClusterFps(fps,cutoff=0.5):
    from rdkit import DataStructs
    from rdkit.ML.Cluster import Butina

    # first generate the distance matrix:
    dists = []
    nfps = len(fps)
    for i in range(1,nfps):
        sims = DataStructs.BulkTanimotoSimilarity(fps[i],fps[:i])
        dists.extend([1-x for x in sims])

    # now cluster the data:
    cs = Butina.ClusterData(dists,nfps,cutoff,isDistData=True)
    return cs

In [ ]:
#getting all fingerprints
fingerprints = []
for index, row in DataSet.iterrows():
    if(DataSet.at[index, 'name']==0):
        continue
    else:
        if len(DataSet.at[index, 'fingerprint'])>4:
            fingerprints.append(DataSet.at[index, 'fingerprint'])
        else:
            continue

In [ ]:
#clustering the fingerprints
clusters=ClusterFps(fingerprints,cutoff=0.5)

In [ ]:
reduced_drugs = []
for i in range(len(clusters)):
    cl = clusters[i]
    id = cl[0]
    reduced_drugs.append(fingerprints[id])

# Preparing data for machinelearning

In [ ]:
ml_data = []
for index, row in DataSet.iterrows():
    if DataSet.at[index, 'fingerprint'] in reduced_drugs:
        if DataSet.at[index, 'target'] in protIds50:
            x=DataSet.at[index,'fingerprint']
            z=embedds[DataSet.at[index, 'target']]
            ml_data.append(np.concatenate((x,z), axis=0))

for entry in ml_data:
    for number in entry:
        number = float(number)
        
np.random.shuffle(ml_data)

In [ ]:
#saving the ml_data as file
from numpy import asarray
from numpy import savez_compressed
savez_compressed('data.npz',ml_data)

In [ ]:
#reading the data form file
from numpy import load
dict_data = load('data.npz')
ml_data = dict_data['arr_0']

In [ ]:
X = []
y = []
for entry in ml_data:
    if(len(entry)==2048):
        X.append(entry)
        y.append(1)

# ML

In [ ]:
#split data into training and test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0, test_size=0.15)
print("Dataset sizes: \nWhole set: {}\nTraining Set: {}\nTest Set: {}".format(len(y), len(y_train), len(y_test)))

In [ ]:
#adding negatives to test data
len_test = len(y_test)
for i in range(len_test):
    fingerprint = random.choice(fingerprints)
    y_test.append(0)
    tgt = random.choice(protIds50)
    seqvec = embedds[tgt]
    temp = np.concatenate((fingerprint,seqvec), axis=0)
    for number in temp:
        number=float(number)
    X_test.append(temp)

In [ ]:
#adding five negatives to the training set
for i in range(5):
    fingerprint= random.choice(fingerprints)
    y_train.append(0)
    tgt = random.choice(protIds50)
    seqvec = embedds[tgt]
    temp = np.concatenate((fingerprint,seqvec),axis=0)
    for number in temp:
        number=float(number)
    X_train.append(temp)


In [ ]:
# Perform cross-validation to optimize hyperparameters

# Define cross-validation object
cv = StratifiedKFold(n_splits = 5)

# Define predictor
from sklearn.svm import SVC
classifier = SVC(probability=True)

# Define parameters we want to optimize and values we want to test
# Here, we test different activation functions
params = { 'decision_function_shape': ['ovo', 'ovr']}

# Perform grid search
grid = GridSearchCV(estimator = classifier, cv = cv, param_grid = params, 
                    return_train_score=True)
grid.fit(X_train, y_train)

# Analyse results

cv_results = pd.DataFrame(grid.cv_results_)
print(cv_results)

In [ ]:
# Use best estimator and assess performance on the test set

# Calculate predictions
best_classifier = grid.best_estimator_
y_pred = best_classifier.predict(X_test)
pred_score = best_classifier.score(X_test, y_test)

# Calculate confusion matrix (showing tp, fp, tn, fn)
cm = confusion_matrix(y_test, y_pred)
print(cm)
print('Acc: {}'.format(round(pred_score, 3)))
predictions=y_pred


In [ ]:
#calculate the AUC for the modell

probas = best_classifier.predict_proba(X_test)
proba_predictions = []
for entry in probas:
    proba_predictions.append(entry[0])
from sklearn.metrics import roc_auc_score
auc = roc_auc_score(y_test, proba_predictions)
print("AUC: {}".format(auc))
AUCs = []
for i in range(len(y_test)):
    truths = []
    preds = []
    for j in range(len(y_test)):
        pick = random.randint(0,len(y_test)-1)
        truth = y_test[pick]
        pred = proba_predictions[pick]
        truths.append(truth)
        preds.append(pred)
    auc2 = roc_auc_score(truths,preds)
    AUCs.append(auc2)
std_auc = np.std(AUCs)
print(std_auc)

In [ ]:
pd.crosstab(y_test, y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

In [ ]:
def calc_std_errs():
    iterations = len(y_test)
    overall_accs = []
    precisions = []
    neg_precs = []
    recalls = []
    neg_covs = []
    f1s = []
    mccs = []
    for i in range(iterations):
        tp = 0
        tn = 0
        fp = 0
        fn = 0
        for j in range(iterations):
            pick = random.randint(0,len(y_test)-1)
            truth = y_test[pick]
            pred = predictions[pick]
            if truth == 1:
                if pred == 1:
                    tp = tp +1
                else:
                    fn = fn +1
            else:
                if pred == 1:
                    fp = fp +1
                else:
                    tn = tn +1
        #formulas of performance mesurements
        precision = tp/(tp+fp)
        precisions.append(precision)
        neg_prec = tn/(tn+fn)
        neg_precs.append(neg_prec)
        recall = tp/(tp+fn)
        recalls.append(recall)
        neg_cov = tn/(tn+fp)
        neg_covs.append(neg_cov)
        f1 = 2*precision*recall/(precision+recall)
        f1s.append(f1)
        overall_acc = (tp+tn)/(tp+fp+tn+fn)
        overall_accs.append(overall_acc)
        mcc = matthews_corrcoef(y_test,predictions)
        mccs.append(mcc)
    #calculate standard deviation of the performance mesurements
    std_prec = np.std(precisions)
    std_neg_prec = np.std(neg_precs)
    std_recall = np.std(recalls)
    std_neg_cov = np.std(neg_covs)
    std_f1 = np.std(f1s)
    std_overall_acc = np.std(overall_accs)
    std_mcc = np.std(mccs)
    print("std_prec: {}\nstd_neg_prec: {}\nstd_recall: {}\nstd_neg_cov: {}\nstd_f1: {}\nstd_overall_acc: {}\nstd_mcc: {}".format(std_prec, std_neg_prec, std_recall, std_neg_cov, std_f1, std_overall_acc, std_mcc))


In [ ]:
tp = 0
tn = 0
fp = 0
fn = 0

for i in range(len(predictions)-1):
    pred = i
    if y_test[i] == 1:
        if pred == 1:
            tp = tp+1
        else:
            fn = fn+1
    else:
        if pred == 1:
            fp = fp + 1
        else:
            tn = tn + 1
print("True Positive: {}\nTrue Negative: {}\nFalse Positive: {}\nFalse Negative: {}".
      format(tp, tn, fp, fn))

In [ ]:
#analysis of the predictions
tp = 1
tn = 685
fp = 1
fn = 685

#calculating performance scores
precision = tp/(tp+fp)
print("Precision:")
print(precision)

neg_prec = tn/(tn+fn)
print("negative Precision:")
print(neg_prec)

recall = tp/(tp+fn)
print("Recall:")
print(recall)

neg_cov = tn/(tn+fp)
print("negative coverage:")
print(neg_cov)

f1 = 2*precision*recall/(precision+recall)
print("f1-score:")
print(f1)

overall_acc = (tp+tn)/(tp+fp+tn+fn)
print("overall accuracy:")
print(overall_acc)

print("MCC:")
print(matthews_corrcoef(y_test,predictions))

calc_std_errs()

In [ ]:
#checking the distirbution of classes in all the data
#data is all the available data
def get_distrib(data):
    positive = 0
    negative = 0
    for entry in data:
        if entry[1] == 1:
            positive = positive + 1
        else:
            negative = negative + 1
    total = positive + negative
    #scaling the data into percentages
    posper = (positive/total) * 100
    return posper



#distribution is the amount of times the dominant class appears out of 100 entrys
def ZeroRuleBaseline(distribution):
    #a random number between 1 and 100 is generated
    tempPred = random.randint(1,101)
    if tempPred <= distribution:
        return 1
    else:
        return 0


In [ ]:
def calc_std_errs_baseline():
    iterations = len(y_test)
    overall_accs = []
    precisions = []
    neg_precs = []
    recalls = []
    neg_covs = []
    f1s = []
    mccs = []
    for i in range(iterations):
        tp = 0
        tn = 0
        fp = 0
        fn = 0
        for j in range(iterations):
            pick = random.randint(0,len(y_test)-1)
            truth = y_test[pick]
            pred = baseline_predicts[pick]
            if truth == 1:
                if pred == 1:
                    tp = tp +1
                else:
                    fn = fn +1
            else:
                if pred == 1:
                    fp = fp +1
                else:
                    tn = tn +1
        #formulas of performance mesurements
        precision = tp/(tp+fp)
        precisions.append(precision)
        neg_prec = tn/(tn+fn)
        neg_precs.append(neg_prec)
        recall = tp/(tp+fn)
        recalls.append(recall)
        neg_cov = tn/(tn+fp)
        neg_covs.append(neg_cov)
        f1 = 2*precision*recall/(precision+recall)
        f1s.append(f1)
        overall_acc = (tp+tn)/(tp+fp+tn+fn)
        overall_accs.append(overall_acc)
        mcc = matthews_corrcoef(y_test,baseline_predicts)
        mccs.append(mcc)
    #calculate standard deviation of the performance mesurements
    std_prec = np.std(precisions)
    std_neg_prec = np.std(neg_precs)
    std_recall = np.std(recalls)
    std_neg_cov = np.std(neg_covs)
    std_f1 = np.std(f1s)
    std_overall_acc = np.std(overall_accs)
    std_mcc = np.std(mccs)
    print("std_prec: {}\nstd_neg_prec: {}\nstd_recall: {}\nstd_neg_cov: {}\nstd_f1: {}\nstd_overall_acc: {}\nstd_mcc: {}".format(std_prec, std_neg_prec, std_recall, std_neg_cov, std_f1, std_overall_acc, std_mcc))


In [ ]:
#calculating the performance of the baseline prediction
dist = get_distrib(ml_data)
tp = 0
tn = 0
fp = 0
fn = 0
baseline_predicts = []
for i in range(len(y_test)):
    pred = ZeroRuleBaseline(dist)
    baseline_predicts.append(pred)
    if y_test[i] == 1:
        if pred == 1:
            tp = tp +1
        else:
            fn = fn +1
    else:
        if pred == 1:
            fp = fp +1
        else:
            tn = tn +1
print("True Positive: {}\nTrue Negative: {}\nFalse Positive: {}\nFalse Negative: {}".format(tp, tn, fp, fn))
#calculating performance scores
precision = tp/(tp+fp)
print("Precision:")
print(precision)

neg_prec = tn/(tn+fn)
print("negative Precision:")
print(neg_prec)

recall = tp/(tp+fn)
print("Recall:")
print(recall)

neg_cov = tn/(tn+fp)
print("negative coverage:")
print(neg_cov)

f1 = 2*precision*recall/(precision+recall)
print("f1-score:")
print(f1)

overall_acc = (tp+tn)/(tp+fp+tn+fn)
print("overall accuracy:")
print(overall_acc)

print("MCC:")
print(matthews_corrcoef(y_test,baseline_predicts))

calc_std_errs_baseline()
